In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import random

# Load the Excel file
SN_df = pd.read_excel('SN_enhancementlist_v2.xlsx')
ADO_df = pd.read_excel('ADO_FeatureUserStorylist_V2.xlsx',skiprows=1)


### Data Wrangling on SN table

In [2]:
np.random.seed(100)

assignees_list = SN_df['Assigned to'].dropna().unique()

nan_assignees = SN_df['Assigned to'].isna()

# Randomly select values from the array to fill NaNs
random_assignees = np.random.choice(assignees_list, size=nan_assignees.sum())

# Replace NaN values in column 'Assigned to' with the randomly selected values
SN_df.loc[nan_assignees, 'Assigned to'] = random_assignees

In [3]:
requestor_list = SN_df['Requested For'].dropna().unique()

nan_requestors = SN_df['Requested For'].isna()

# Randomly select values from the array to fill NaNs
random_requestors = np.random.choice(requestor_list, size=nan_requestors.sum())

# Replace NaN values in column 'Assigned to' with the randomly selected values
SN_df.loc[nan_requestors, 'Requested For'] = random_requestors

In [4]:
# Check what the median completion time of a ticket looks like
closed_SN = SN_df.query('State == "Closed Complete"')
completion_times = closed_SN['Opened'] - closed_SN['Updated']
completion_times.median()

Timedelta('-96 days +02:28:19')

In [5]:
# Impute 'Due date' with a normal distribution with mean = 90 days from opened date

random_days = np.random.normal(loc=90, scale=50, size=len(SN_df))
SN_df['days_delta'] = random_days
SN_df.loc[:,'Due date'] = SN_df.loc[:,'Opened'] + pd.to_timedelta(random_days, unit='D');


/var/folders/dp/3qfg9zkn0274x0bm9wcgqjnw0000gq/T/ipykernel_85594/1164315892.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2025-04-09 20:33:35.352502073', '2025-02-07 10:16:45.769697739',
 '2025-02-19 11:03:35.485337699', '2025-03-21 07:09:37.442607284',
 '2025-01-13 05:28:21.988925237', '2024-09-24 06:04:06.179022609',
 '2024-07-30 04:39:53.669415498', '2024-05-28 10:06:57.262467848',
 '2024-06-09 04:40:56.367255679', '2024-06-09 08:56:53.637761993',
 ...
 '2023-10-28 00:18:03.412526254', '2024-06-03 02:15:26.730302012',
 '2024-08-13 16:53:46.552376654', '2024-07-23 12:50:34.733871297',
 '2024-06-16 22:31:26.625042379', '2024-07-07 12:28:29.899520367',
 '2024-08-07 05:57:27.219696556', '2024-07-04 14:03:51.785581851',
 '2024-02-21 23:04:09.092698960', '2024-07-18 05:39:17.990839643']
Length: 122, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatib

In [6]:
# Download the stopwords dataset
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a text string
def remove_stopwords(text):
    if isinstance(text, str):  # Only process strings
        # Split the text into words, remove stopwords, and join the words back
        return ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text  # Return the value unchanged if it's not a string

# Apply the function to the 'text' column
SN_df['Cleaned description'] = SN_df['Short description'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameer.chowdary.gaddam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Wrangling on ADO table

In [7]:

# Replace missing values in column 'Story Points (0 - 99)' with the randomly selected values
ADO_df['Story Points (0 - 99)'] = ADO_df['Story Points (0 - 99)'].apply(lambda x: int(np.random.uniform(1, 99)) if pd.isna(x) else x)

In [8]:
# Function to categorize level of effort based on story points
def categorize_effort(points):
    if points <= 33:
        return 'low'
    elif points <= 66:
        return 'medium'
    else:
        return 'high'

# Apply the function to the 'points' column to create 'effort' column
ADO_df['Level of Effort'] = ADO_df['Story Points (0 - 99)'].apply(categorize_effort)

In [9]:
# Replace missing values in column 'Priority (0 - 4)' with randomly selected values
ADO_df['Priority (0 - 4)'] = ADO_df['Priority (0 - 4)'].apply(lambda x: int(np.random.uniform(0, 5)) if pd.isna(x) else x)

In [10]:
# Apply the function to the 'Feature Title' column
ADO_df['Cleaned Feature Title'] = ADO_df['Feature Title'].apply(remove_stopwords)

# Apply the function to the 'text' column
ADO_df['Cleaned User Story Title'] = ADO_df['User Story Title'].apply(remove_stopwords)


In [11]:
# Remove leading backslash from 'Iteration Path' values
ADO_df['Iteration Path'] = ADO_df['Iteration Path'].str.lstrip('\\')

# Set 'Target Date' to November 30th for rows where 'Iteration Path' is 'Poppy'
ADO_df.loc[ADO_df['Iteration Path'] == 'Poppy', 'Target Date'] = pd.to_datetime('2024-11-30')

# Set 'Target Date' to December 15, 2024, for rows where 'Iteration Path' is 'Carnation'
ADO_df.loc[ADO_df['Iteration Path'] == 'Carnation', 'Target Date'] = pd.to_datetime('2024-12-15')

# Filter for active cases where 'Iteration Path' is empty or null
active_no_iteration_path = ADO_df[(ADO_df['State'] == 'Active') & (ADO_df['Iteration Path'].isnull() | (ADO_df['Iteration Path'] == ''))]

# Define the Iteration Paths and their corresponding Target Dates
iteration_paths = {
    'Rose': '2025-01-04',
    'Lily': '2025-01-25',
    'Marigold': '2025-02-15',
    'Snapdragon': '2025-03-08',
    'Bluebell': '2025-03-29',
    'Anemone': '2025-04-30'  # Assume end of April for ongoing cases
}

# Convert Target Dates to datetime format
iteration_paths = {key: pd.to_datetime(value) for key, value in iteration_paths.items()}

# Repeat Iteration Paths to match the number of active cases needing assignment
assignable_paths = list(iteration_paths.items()) * (len(active_no_iteration_path) // len(iteration_paths)) + random.sample(list(iteration_paths.items()), len(active_no_iteration_path) % len(iteration_paths))

# Randomly shuffle and assign each Iteration Path and Target Date to the active cases without an Iteration Path
random.shuffle(assignable_paths)
ADO_df.loc[active_no_iteration_path.index, ['Iteration Path', 'Target Date']] = assignable_paths[:len(active_no_iteration_path)]

# Define the Iteration Paths and their corresponding Target Dates for closed cases
closed_iteration_paths = {
    'Hydrangea': '2024-03-23',
    'Dahlia': '2024-04-13',
    'Sunflower': '2024-05-04',
    'Jasmine': '2024-05-25',
    'Zinnia': '2024-06-15',
    'Daffodil': '2024-07-06',
    'Lavender': '2024-07-27',
    'Chrysanthemum': '2024-08-17',
    'Hibiscus': '2024-09-07',
    'Gardenia': '2024-09-28',
    'Orchid': '2024-10-19',
    'Tulip': '2024-11-09'
}

# Convert Target Dates to datetime format
closed_iteration_paths = {key: pd.to_datetime(value) for key, value in closed_iteration_paths.items()}

# Assign 'Target Date' for closed cases based on 'Iteration Path'
for path, target_date in closed_iteration_paths.items():
    ADO_df.loc[(ADO_df['State'] == 'Closed') & (ADO_df['Iteration Path'] == path), 'Target Date'] = target_date

# Display the updated rows to confirm changes
print("Updated closed cases with assigned Target Dates:")
print(ADO_df[(ADO_df['State'] == 'Closed') & (ADO_df['Iteration Path'].isin(closed_iteration_paths.keys()))][['Iteration Path', 'Target Date']])

# Define the Iteration Paths and their corresponding Target Dates for closed cases, including Poppy
closed_iteration_paths = {
    'Hydrangea': '2024-03-23',
    'Dahlia': '2024-04-13',
    'Sunflower': '2024-05-04',
    'Jasmine': '2024-05-25',
    'Zinnia': '2024-06-15',
    'Daffodil': '2024-07-06',
    'Lavender': '2024-07-27',
    'Chrysanthemum': '2024-08-17',
    'Hibiscus': '2024-09-07',
    'Gardenia': '2024-09-28',
    'Orchid': '2024-10-19',
    'Tulip': '2024-11-09',
    'Poppy': '2024-11-30'  # Adding Poppy with its assigned end date
}

# Convert Target Dates to datetime format
closed_iteration_paths = {key: pd.to_datetime(value) for key, value in closed_iteration_paths.items()}

# List of paths and dates for random assignment
iteration_path_dates = list(closed_iteration_paths.items())

# Filter for closed cases without a Target Date assigned
closed_no_target_date = ADO_df[(ADO_df['State'] == 'Closed') & (ADO_df['Target Date'].isnull())]

# Randomly assign each row an Iteration Path and Target Date from the list
assigned_paths = random.choices(iteration_path_dates, k=len(closed_no_target_date))
ADO_df.loc[closed_no_target_date.index, ['Iteration Path', 'Target Date']] = assigned_paths



Updated closed cases with assigned Target Dates:
    Iteration Path Target Date
4        Hydrangea  2024-03-23
7        Hydrangea  2024-03-23
10          Dahlia  2024-04-13
24       Sunflower  2024-05-04
26       Hydrangea  2024-03-23
33       Hydrangea  2024-03-23
35       Hydrangea  2024-03-23
43       Sunflower  2024-05-04
44         Jasmine  2024-05-25
45         Jasmine  2024-05-25
47          Zinnia  2024-06-15
48          Zinnia  2024-06-15
54        Lavender  2024-07-27
57        Daffodil  2024-07-06
61        Lavender  2024-07-27
72          Zinnia  2024-06-15
74       Hydrangea  2024-03-23
75        Lavender  2024-07-27
102  Chrysanthemum  2024-08-17
104       Hibiscus  2024-09-07
105       Hibiscus  2024-09-07
106       Gardenia  2024-09-28
107       Gardenia  2024-09-28
108         Orchid  2024-10-19
109       Gardenia  2024-09-28
110         Orchid  2024-10-19
111        Jasmine  2024-05-25
112        Jasmine  2024-05-25
114       Hibiscus  2024-09-07
115         Dahlia  2

### Data export

In [12]:
SN_df.to_excel('SN_output.xlsx', index=False)
ADO_df.to_excel('ADO_output.xlsx', index=False)